## Part 1: Document Loading

Type of documents covered:
- PDFs
- Youtube Videos
- Website 

Import and stardardise such that we obtain:
- Content
- Meta data

In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

### PDF

Each page is a Document. A Document contains text (page_content) and metadata.

In [ ]:
# pip install pypdf 

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("C:/Users/richi/OneDrive/Documents/OpenAI API practice/Langchain for LLM applications/Deep_Learning_A4.pdf")
pages = loader.load()

In [ ]:
page = pages[0]
page.page_content[0:500] # number of characters
page.metadata # 

### Youtube (Broken - to be fixed)
- Issue: yt_dlp can't find the ffmpeg files, even though they're properly installed on the local device. Didn't resolve why yet...
- Tutorial used: https://www.youtube.com/watch?v=IECI72XEox0

In [ ]:
# ! pip install yt_dlp
# ! pip install pydub

In [ ]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser # converts youtube audio to text format (langchain model)
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

In [ ]:
# Youtube URL (video: Josh Angrist: What's the Difference Between Econometrics and Data Science? - 2 min)
url = "https://www.youtube.com/watch?v=2EhRT2mOXm8&t=2s"

# Directory where to save audio
save_dir = "C:/Users/richi/OneDrive/Documents/OpenAI API practice/Langchain for LLM applications/"


In [ ]:
# DOESN'T WORK FOR SOME REASON - installed fmpeg stuff...

# Note: may take a while & will give error if the content is already present/downloaded in file directory
loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser()
)

docs = loader.load()

In [ ]:
doc = docs[0]
doc.page_content[0: 500]
doc.metadata

### URLs

Note: format is probably really poorly formatted, so we should post-process for readability.

In [ ]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://richie-lee.github.io/post/2021_uplift/")

In [ ]:
docs = loader.load()

In [ ]:
doc = docs[0]
doc.page_content[:1500]

---
## Part 2: Document splitting

*After* loading data and *before* feeding it into the vector store 

Fundamental concept: splitting on chunks with some size, with overlap. This overlap is helpful in ensuring no information is loss when splitting texts.

Types of splitting:
- **CharacterTextSplitter():** based on characters
- **MarkdownHeaderTextSplitter():** based on MD headers
- **TokenTextSplitter():** based on tokens
- **RecursiveCharacterTextSplitter():** recursively tries to split by different characters to see what works
- **Language():** for Python, Ruby, Markdown, ...
- **NLTKTextSplitter():** based on sentences and NLTK (natural language tool kit)
- **SpacyTextSplitter():** based on sentences and Spacy




In [ ]:
import openai
import os

# Connecting to account
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv("C:/Users/richi/OneDrive/Documents/OpenAI API practice/openai_api_key.env")) # read local .env file
openai.api_key = os.environ["OPENAI_API_KEY"]

Intuitive examples of (Recursive) character text splitters:

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter

In [ ]:
chunk_size =26
chunk_overlap = 4

In [ ]:
# initialise two different text splitters
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap
)

In [ ]:
# n < 26 (chunk size)
text1 = 'abcdefghijklmnopqrstuvwxyz'
print(r_splitter.split_text(text1))

# n > 26 (chunk size)
text2 = 'abcdefghijklmnopqrstuvwxyzabcdefg'
print(r_splitter.split_text(text2))

character text splitting issue: it splits on a new characters, by default a newline char, but here there arent't any.

In [ ]:
text3 = "a b c d e f g h i j k l m n o p q r s t u v w x y z"
print(r_splitter.split_text(text3)) # recursive character text splitting
print(c_splitter.split_text(text3)) # character text splitting: issue it splits on a new characters, by default a newline char, but here there arent't any.

In [ ]:
# Note - given processing C & R become equivalent
c_splitter = CharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    separator = ' '
)
c_splitter.split_text(text3)

Recursive splitting details:

In [ ]:
some_text = """When writing documents, writers will use document structure to group content. \
This can convey to the reader, which idea's are related. For example, closely related ideas \
are in sentances. Similar ideas are in paragraphs. Paragraphs form a document. \n\n  \
Paragraphs are often delimited with a carriage return or two carriage returns. \
Carriage returns are the "backslash n" you see embedded in this string. \
Sentences have a period at the end, but also, have a space.\
and words are separated by space."""

In [ ]:
len(some_text)

In [ ]:
c_splitter = CharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0,
    separator = ' '
)
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""] # default sepearators, but here for illustration explicitly displayed - it moves from left to right recursively
)

In [ ]:
# Only splits on spaces
c_splitter.split_text(some_text)

In [ ]:
# Splits on \n\n first, and then rest respectively for better quality due to importance hierarchy
r_splitter.split_text(some_text)

For periods, define regex with lookback for better results: "(?<=\.)"

In [ ]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150,
    chunk_overlap=0,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""] # separators=["\n\n", "\n", "\. ", " ", ""] not this due to REGEX under the hood
)
r_splitter.split_text(some_text)

Try with real example:

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("C:/Users/richi/OneDrive/Documents/OpenAI API practice/Langchain for LLM applications/Deep_Learning_A4.pdf")
pages = loader.load()

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,
    chunk_overlap=150,
    length_function=len
)

In [ ]:
docs = text_splitter.split_documents(pages)

In [ ]:
# To illustrate difference it may make
print(len(docs), len(pages))

**Token splitting:** LLMs often have context windows designated in tokens (approx 4 characters often).

In [ ]:
from langchain.text_splitter import TokenTextSplitter

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=1, chunk_overlap=0)

In [ ]:
text1 = "foo bar bazzyfoo"

In [ ]:
text_splitter.split_text(text1)

In [ ]:
text_splitter = TokenTextSplitter(chunk_size=10, chunk_overlap=0)

In [ ]:
docs = text_splitter.split_documents(pages)

In [ ]:
# Note metadata is same in chunk as in pages (which is good).
docs[0]
pages[0].metadata

**Context aware splitting:** adds meta data to the text chunks

- chunks aim to keep text with common context together
- text splitting often uses sentences or other delimiters to keep related text together, but some docs have explicit structures that can be used (e.g. markdown headers) - headers become metadata

In [ ]:
from langchain.document_loaders import NotionDirectoryLoader
from langchain.text_splitter import MarkdownHeaderTextSplitter

In [ ]:
markdown_document = """# Title\n\n \
## Chapter 1\n\n \
Hi this is Jim\n\n Hi this is Joe\n\n \
### Section \n\n \
Hi this is Lance \n\n 
## Chapter 2\n\n \
Hi this is Molly"""

In [ ]:
headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

In [ ]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=headers_to_split_on
)
md_header_splits = markdown_splitter.split_text(markdown_document)

In [ ]:
print(len(md_header_splits)) # number chunks

print(md_header_splits[0]) # first chunk